# MIRAGE++ Notebook 4: Application to Real Financial Data

In this notebook, we'll use MIRAGE++ on real financial data. We'll download stock prices, engineer features, fit the model, and compare it to OLS, Ridge, and Lasso. Every step is explained for clarity.

## 1. Downloading Real Financial Data

We'll use the `yfinance` package to get daily stock prices for several assets. If you don't have it, install with `!pip install yfinance`.

In [ ]:
import yfinance as yf
import pandas as pd
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META']
data = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Adj Close']
data = data.dropna()
data.head()

## 2. Feature Engineering: Lagged Returns and Technical Indicators

We'll create features like lagged returns and moving averages for each stock.

In [ ]:
returns = data.pct_change().dropna()
features = []
for lag in range(1, 6):
    features.append(returns.shift(lag))
ma = data.rolling(5).mean().pct_change().dropna()
features.append(ma)
X = pd.concat(features, axis=1).dropna()
X = X.loc[X.index.intersection(returns.index)]
y = returns.mean(axis=1).loc[X.index]  # Predict average return across assets
print('Feature matrix shape:', X.shape)

## 3. Standardize Features

It's good practice to standardize features for regression.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = y.values
print('Standardized feature matrix shape:', X_scaled.shape)

## 4. Fit MIRAGE++ to the Real Data

We'll use the same MIRAGE++ code as before.

In [ ]:
def entropy(theta, epsilon=1e-8):
    theta = np.clip(theta, epsilon, 1.0)
    return -np.sum(theta * np.log(theta))

def loss(X, y, theta, lam):
    preds = X @ theta
    mse = np.mean((preds - y)**2)
    ent = entropy(theta)
    return mse + lam * ent

def gradient(X, y, theta, lam):
    preds = X @ theta
    grad_mse = 2 * X.T @ (preds - y) / len(y)
    grad_entropy = -1 - np.log(np.clip(theta, 1e-8, 1.0))
    return grad_mse + lam * grad_entropy

def mirror_descent_step(grad, theta_t, eta):
    theta_new = theta_t * np.exp(-eta * grad)
    theta_new = np.clip(theta_new, 1e-12, None)
    return theta_new / np.sum(theta_new)

def fit_mirage(X, y, lam=0.1, eta=0.2, n_iters=300):
    n = X.shape[1]
    theta = np.ones(n) / n
    loss_hist = []
    for i in range(n_iters):
        grad = gradient(X, y, theta, lam)
        theta = mirror_descent_step(grad, theta, eta)
        loss_hist.append(loss(X, y, theta, lam))
    return theta, loss_hist

theta_mirage, loss_hist = fit_mirage(X_scaled, y, lam=0.1, eta=0.2, n_iters=300)
print('MIRAGE++ weights:', theta_mirage)

## 5. Compare to OLS, Ridge, and Lasso

Let's fit standard models and compare their weights and prediction errors.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
ols = LinearRegression().fit(X_scaled, y)
ridge = Ridge(alpha=0.1).fit(X_scaled, y)
lasso = Lasso(alpha=0.1).fit(X_scaled, y)

plt.figure(figsize=(12,4))
plt.bar(np.arange(X_scaled.shape[1])-0.3, ols.coef_, width=0.2, label='OLS')
plt.bar(np.arange(X_scaled.shape[1])-0.1, ridge.coef_, width=0.2, label='Ridge')
plt.bar(np.arange(X_scaled.shape[1])+0.1, lasso.coef_, width=0.2, label='Lasso')
plt.bar(np.arange(X_scaled.shape[1])+0.3, theta_mirage, width=0.2, label='MIRAGE++')
plt.xlabel('Feature Index')
plt.ylabel('Weight')
plt.title('Model Weights Comparison (Real Data)')
plt.legend()
plt.show()

## 6. Prediction Error Comparison

Let's compare mean squared error (MSE) for each model.

In [ ]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

y_pred_ols = ols.predict(X_scaled)
y_pred_ridge = ridge.predict(X_scaled)
y_pred_lasso = lasso.predict(X_scaled)
y_pred_mirage = X_scaled @ theta_mirage

print('OLS MSE:', mse(y, y_pred_ols))
print('Ridge MSE:', mse(y, y_pred_ridge))
print('Lasso MSE:', mse(y, y_pred_lasso))
print('MIRAGE++ MSE:', mse(y, y_pred_mirage))

## 7. Sharpe Ratio Comparison

The Sharpe ratio measures risk-adjusted return. Let's compare it for each model's predictions.

In [ ]:
def sharpe_ratio(returns):
    return np.mean(returns) / np.std(returns)

print('OLS Sharpe:', sharpe_ratio(y_pred_ols))
print('Ridge Sharpe:', sharpe_ratio(y_pred_ridge))
print('Lasso Sharpe:', sharpe_ratio(y_pred_lasso))
print('MIRAGE++ Sharpe:', sharpe_ratio(y_pred_mirage))

## 8. Interpretability and Diversity

MIRAGE++ weights are positive and sum to 1, making them easy to interpret as allocations or probabilities.

Let's check this property.

In [ ]:
print('MIRAGE++ weights (should be positive):', theta_mirage)
print('Sum of MIRAGE++ weights:', np.sum(theta_mirage))

## 9. Summary

- MIRAGE++ works on real financial data and produces interpretable, diversified weights.
- It performs competitively with OLS, Ridge, and Lasso.
- Weights are always positive and sum to 1.

In the next notebook, we'll explore advanced geometry and extensions!